# Script to analyse UKBB whole brain data in the WMHOB project

## Load packages

In [2]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)

# Load data and exclude participants
Exclusion criteria:
- Bipolar disorder
- Personality disorders
- Schizophrenia
- Depression
- Mania
- Bulimia nervosa
- Anorexia nervosa
- ADD/ADHD
- Panic attacks
- Diabetes
- Hypertension
- Neurological disorders
- Underweight (BMI<18.5)

In [3]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$schizophrenia[di[i]=="Depression"] <- 1
di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
di$schizophrenia[di[i]=="Panic attacks"] <- 1
}


# Exclude participants who have diabetes
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$diabetes[di[i] =="Yes"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
#di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1
#di$excluded[di$diabetes ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')
nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0*
                    ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0)
ukbb_all$logBMI3=log(ukbb_all$body_mass_index_bmi_f21001_2_0)
ukbb_all$WHR=ukbb_all$waist_circumference_f48_2_0/ukbb_all$hip_circumference_f49_2_0

#Remove outliers
variables_for_outliers=c(58:88, 120:150, 26:27, 32:38, 42:43, 48:54, 89:119, 
                         151:181, 9, 2287:2300, 2116, 5032:5034) # all volume and thickness variables from FS, BMI, eTIV, area

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_all[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_all[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_all[j][ukbb_all[j]<lower]=NA
    ukbb_all[j][ukbb_all[j]>upper]=NA
}

ukbb_all=ukbb_all[complete.cases(ukbb_all[,5032:5034]),] # BMI, WMH, WHR
nrow(ukbb_all)

ukbb_all$Hypertension=0
ukbb_all$Hypertension[ukbb_all$vascular_heart_diagnoses=='1']=1
ukbb_all$Diabetes=0
ukbb_all$Diabetes[ukbb_all$diabetes=='1']=1

# Create separate dataframes for each imaging feature set:
- cortical thickness
- cortical volume
- subcortical volume as derived from FSL
- surface area

In [9]:
ukbb_ct=ukbb_all[complete.cases(ukbb_all[,c(58:88, 120:150)]),] # CT
ukbb_ctxvol=ukbb_all[complete.cases(ukbb_all[,c(89:119, 151:181, 9)]),]
ukbb_subvol=ukbb_all[complete.cases(ukbb_all[,c(2287:2300, 2116)]),]
ukbb_area=ukbb_all[complete.cases(ukbb_all[,c(grep('^area_of.*_2_',colnames(ukbb_all)))]),]

In [10]:
nrow(ukbb_ct)
nrow(ukbb_ctxvol)
nrow(ukbb_subvol)
nrow(ukbb_area)

## Save file with outliers and clinical groups excluded

In [11]:
write.table(ukbb_all, file = "/dagher/dagher11/filip/OBAL/output/UKBB_preprocessed.csv", 
            append = FALSE, quote = FALSE, sep = ",", na = "NA", dec = ".", row.names = FALSE,
            col.names = TRUE)

## BMI and subcortical volumes

In [23]:
StatssubvolFSL_BMI=matrix(nrow=length(c(2287:2300)),ncol=3)
index=1
for (i in c(2287:2300)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_subvol[[i]]*ukbb_subvol[[2116]])-mean(ukbb_subvol[[i]]*ukbb_subvol[[2116]]))/
    sd(ukbb_subvol[[i]]*ukbb_subvol[[2116]])
    
    b=(summary(lm(scaled ~ 
                     scale(log(ukbb_subvol[[1891]])) +
                     scale(ukbb_subvol[[1897]]) +
                     as.factor(ukbb_subvol[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    StatssubvolFSL_BMI[index,][2]=b$coefficients[2,][4]
    StatssubvolFSL_BMI[index,][1]=colnames(ukbb_subvol)[i]
    index=index+1
    
}

StatssubvolFSL_BMI_t=matrix(nrow=length(c(2287:2300)),ncol=2)
index=1
for (i in c(2287:2300)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_subvol[[i]]*ukbb_subvol[[2116]])-mean(ukbb_subvol[[i]]*ukbb_subvol[[2116]]))/
    sd(ukbb_subvol[[i]]*ukbb_subvol[[2116]], na.rm=TRUE)
    
    b=(summary(lm(scaled ~ 
                     scale(log(ukbb_subvol[[1891]])) +
                     scale(ukbb_subvol[[1897]]) +
                     as.factor(ukbb_subvol[[185]]))))
    
    StatssubvolFSL_BMI_t[index,][2]=b$coefficients[2,][3]
    StatssubvolFSL_BMI_t[index,][1]=colnames(ukbb_subvol)[i]
    index=index+1
    
}

In [24]:
StatssubvolFSL_BMI_t[p.adjust(StatssubvolFSL_BMI[,2], method='BH')<0.05]

In [25]:
print(StatssubvolFSL_BMI)

In [26]:
write.table(p.adjust(StatssubvolFSL_BMI[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/FSL_BMI_pvalue.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table((StatssubvolFSL_BMI_t[,2]),'/dagher/dagher11/filip/OBAL/output/FSL_BMI_tvalue.csv',sep=',',quote=FALSE, row.names=FALSE)


## BMI and cortical volume

In [52]:
ukbb_ctxvol$a2=ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0*ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0

In [56]:
StatsctxvolBMI=matrix(nrow=length(c(89:119, 151:181)),ncol=2)
index=1
for (i in c(89:119, 151:181)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])-mean(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]]))/sd(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])
    
    c=(summary(lm(scaled ~ 
                     scale(ukbb_ctxvol$logBMI3) +
                     scale(ukbb_ctxvol[[1897]]) +
                     as.factor(ukbb_ctxvol[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    StatsctxvolBMI[index,][2]=c$coefficients[2,][4]
    StatsctxvolBMI[index,][1]=colnames(ukbb_ctxvol)[i]
    index=index+1
    
}

StatsctxvolBMI_t=matrix(nrow=length(c(89:119, 151:181)),ncol=2)
index=1
for (i in c(89:119, 151:181)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])-mean(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]]))/sd(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])
    
    c=(summary(lm(scaled ~ 
                     scale(ukbb_ctxvol$logBMI3) +
                     scale(ukbb_ctxvol[[1897]]) +
                     as.factor(ukbb_ctxvol[[185]]))))
    
    StatsctxvolBMI_t[index,][2]=c$coefficients[2,][3]
    StatsctxvolBMI_t[index,][1]=colnames(ukbb_ctxvol)[i]
    index=index+1
    
}

In [57]:
write.table(p.adjust(StatsctxvolBMI[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainvolctxBMI_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(StatsctxvolBMI_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainvolctxBMI_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

In [58]:
print(StatsctxvolBMI_t[p.adjust(StatsctxvolBMI[,2], method = 'BH')<0.05])

## Thickness and BMI

In [65]:
Statsctbmi=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ct[[i]])-mean(ukbb_ct[[i]]))/sd(ukbb_ct[[i]])
    
    d=(summary(lm(scaled ~ 
                     scale(log(ukbb_ct[[1891]]))+
                     scale(ukbb_ct[[1897]]) + 
                     as.factor(ukbb_ct[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    Statsctbmi[index,][2]=d$coefficients[2,][4]
    Statsctbmi[index,][1]=colnames(ukbb_ct)[i]
    index=index+1
    
}

Statsctbmi_t=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ct[[i]])-mean(ukbb_ct[[i]]))/sd(ukbb_ct[[i]])
    
    d=(summary(lm(scaled ~ 
                     scale(log(ukbb_ct[[1891]]))+
                     scale(ukbb_ct[[1897]]) +
                     as.factor(ukbb_ct[[185]]))))
    
    Statsctbmi_t[index,][2]=d$coefficients[2,][3]
    Statsctbmi_t[index,][1]=colnames(ukbb_ct)[i]
    index=index+1
    
}

In [66]:
write.table(p.adjust(Statsctbmi[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainBMI_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statsctbmi_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainBMI_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

In [64]:
print(Statsctbmi_t[p.adjust(Statsctbmi[,2], method = 'BH')<0.05])

In [34]:
write.table(p.adjust(Statsareabmi[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainBMIarea_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Statsareabmi_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainBMIarea_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Waist to hip ratio

## CT

In [67]:
StatsctWHR=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ct[[i]])-mean(ukbb_ct[[i]]))/sd(ukbb_ct[[i]])
    
    d=(summary(lm(scaled ~ 
                     scale(ukbb_ct$WHR) +
                     scale(ukbb_ct[[1897]]) +
                     as.factor(ukbb_ct[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    StatsctWHR[index,][2]=d$coefficients[2,][4]
    StatsctWHR[index,][1]=colnames(ukbb_ct)[i]
    index=index+1
    
}

StatsctWHR_t=matrix(nrow=length(c(58:88, 120:150)),ncol=2)
index=1
for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ct[[i]])-mean(ukbb_ct[[i]]))/sd(ukbb_ct[[i]])
    
    d=(summary(lm(scaled ~ 
                     scale(ukbb_ct$WHR) +
                     scale(ukbb_ct[[1897]]) +
                     as.factor(ukbb_ct[[185]]))))
    
    StatsctWHR_t[index,][2]=d$coefficients[2,][3]
    StatsctWHR_t[index,][1]=colnames(ukbb_ct)[i]
    index=index+1
    
}

In [68]:
write.table(p.adjust(StatsctWHR[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainCTWHR_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(StatsctWHR_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainCTWHR_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Cortical volume

In [69]:
StatsvolWHR=matrix(nrow=length(c(89:119, 151:181)),ncol=2)
index=1
for (i in c(89:119, 151:181)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])-mean(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]]))/sd(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])
    
    c=(summary(lm(scaled ~ 
                     scale(ukbb_ctxvol$WHR) +
                     scale(ukbb_ctxvol[[1897]]) +
                     as.factor(ukbb_ctxvol[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    StatsvolWHR[index,][2]=c$coefficients[2,][4]
    StatsvolWHR[index,][1]=colnames(ukbb_ctxvol)[i]
    index=index+1
    
}

StatsvolWHR_t=matrix(nrow=length(c(89:119, 151:181)),ncol=2)
index=1
for (i in c(89:119, 151:181)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])-mean(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]]))/sd(ukbb_ctxvol[[i]]/ukbb_ctxvol[[9]])
    
    c=(summary(lm(scaled ~ 
                     scale(ukbb_ctxvol$WHR) +
                     scale(ukbb_ctxvol[[1897]]) +
                     as.factor(ukbb_ctxvol[[185]]))))
    
    StatsvolWHR_t[index,][2]=c$coefficients[2,][3]
    StatsvolWHR_t[index,][1]=colnames(ukbb_ctxvol)[i]
    index=index+1
    
}

In [70]:
write.table(p.adjust(StatsvolWHR[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/wholebrainVOLWHR_pval.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(StatsvolWHR_t[,2],'/dagher/dagher11/filip/OBAL/output/wholebrainVOLWHR_tval.csv',sep=',',quote=FALSE, row.names=FALSE)

## Subcortical volumes

In [71]:
StatssubvolFSL_WHR=matrix(nrow=length(c(2287:2300)),ncol=2)
index=1
for (i in c(2287:2300)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_subvol[[i]]*ukbb_subvol[[2116]])-mean(ukbb_subvol[[i]]*ukbb_subvol[[2116]]))/sd(ukbb_subvol[[i]]*ukbb_subvol[[2116]])
    
    b=(summary(lm(scaled ~ 
                     scale(ukbb_subvol$WHR) +
                     scale(ukbb_subvol[[1897]]) +
                     as.factor(ukbb_subvol[[185]]), center=FALSE, qr = TRUE, maxIter=100000, Ca=0.0001)))
    
    StatssubvolFSL_WHR[index,][2]=b$coefficients[2,][4]
    StatssubvolFSL_WHR[index,][1]=colnames(ukbb_subvol)[i]
    index=index+1
    
}

StatssubvolFSL_WHR_t=matrix(nrow=length(c(2287:2300)),ncol=2)
index=1
for (i in c(2287:2300)) { #All thickness variables 58:88, 120:150
    
    scaled=((ukbb_subvol[[i]]*ukbb_subvol[[2116]])-mean(ukbb_subvol[[i]]*ukbb_subvol[[2116]]))/sd(ukbb_subvol[[i]]*ukbb_subvol[[2116]])
    
    b=(summary(lm(scaled ~ 
                     scale(ukbb_subvol$WHR) +
                     scale(ukbb_subvol[[1897]]) +
                     as.factor(ukbb_subvol[[185]]))))
    
    StatssubvolFSL_WHR_t[index,][2]=b$coefficients[2,][3]
    StatssubvolFSL_WHR_t[index,][1]=colnames(ukbb_subvol)[i]
    index=index+1
    
}

In [72]:
StatssubvolFSL_WHR_t[p.adjust(StatssubvolFSL_WHR[,2], method='BH')<0.05]

In [73]:
write.table(p.adjust(StatssubvolFSL_WHR[,2], method = 'BH'),'/dagher/dagher11/filip/OBAL/output/FSL_WHR_pvalue.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table((StatssubvolFSL_WHR_t[,2]),'/dagher/dagher11/filip/OBAL/output/FSL_WHR_tvalue.csv',sep=',',quote=FALSE, row.names=FALSE)
